<a href="https://colab.research.google.com/github/Antonino-Marques-Jares/EMBEDDING_ACORDAOS_TCEAC/blob/main/Embeddings_Acordaos_TCE_AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
!pip install -q -U google-generativeai
 # Import the Python SDK
import google.generativeai as genai

import numpy as np
import pandas as pd

import glob


# Apresentando a chave para acesso do geminai

In [80]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Listando os modelos do gemini disponíveis

In [81]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


# Vamos trabalhar com Acórdãos do Tribunal de Contas do Estado do Acre.
O DataSet foi criado no projeto do GitHub [Leitura de documentos](https://github.com/Antonino-Marques-Jares/Projeto_Leitura_Documentos) onde eu também tive participação na extração de dados.



In [82]:
acordaos = pd.read_csv(f"/content/acordaos03.csv",index_col=[0])
acordaos.head()

,conteudo,acordao,dia
0,relator conselheiro marciliano reis fleming v...,176,02/04/1992
1,natureza apenso prestação contas prefeitura m...,7556,02/02/2012
2,natureza tce prestação contas câmara municipa...,7557,02/02/2012
3,natureza anexos prestação contas empresa muni...,7560,02/02/2012
4,natureza tce relatório gestão fiscal prefeitu...,7564,02/02/2012


In [83]:
acordaos['embedding']=''
acordaos.head()

,conteudo,acordao,dia,embedding
0,relator conselheiro marciliano reis fleming v...,176,02/04/1992,
1,natureza apenso prestação contas prefeitura m...,7556,02/02/2012,
2,natureza tce prestação contas câmara municipa...,7557,02/02/2012,
3,natureza anexos prestação contas empresa muni...,7560,02/02/2012,
4,natureza tce relatório gestão fiscal prefeitu...,7564,02/02/2012,


In [85]:
model = "models/embedding-001"
def embed_fn(text):
  return genai.embed_content(model="models/embedding-001", content=text, task_type="RETRIEVAL_DOCUMENT")["embedding"]

acordaos['embedding'] = acordaos.apply(lambda row: embed_fn(row["conteudo"]), axis=1)
acordaos.head()

,conteudo,acordao,dia,embedding
0,relator conselheiro marciliano reis fleming v...,176,02/04/1992,"[0.044043534, -0.013633516, -0.08233992, 0.039..."
1,natureza apenso prestação contas prefeitura m...,7556,02/02/2012,"[0.012121538, 0.010943754, -0.01990369, 0.0321..."
2,natureza tce prestação contas câmara municipa...,7557,02/02/2012,"[-0.0030481075, 0.023675635, -0.01741148, 0.03..."
3,natureza anexos prestação contas empresa muni...,7560,02/02/2012,"[0.023992693, -0.00030531723, -0.03027018, 0.0..."
4,natureza tce relatório gestão fiscal prefeitu...,7564,02/02/2012,"[0.019929623, -0.010757494, -0.040829554, 0.02..."


In [88]:
acordaos.to_csv("\\content\\embeddings.csv")

In [96]:
def gerar_e_buscar_consulta(consulta,model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(acordaos["embedding"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return acordaos.iloc[indice]["conteudo"]

model = "models/embedding-001"
consulta = "Segurança Pública"
gerar_e_buscar_consulta(consulta,model)

' plenário pág avenida ceará bairro bec rio branco acre cep telefone fonefax email prestceacgovbr entidade secretaria estado segurança pública sesp natureza prestação contas objeto prestação contas secretaria estado segurança pública sesp referente exercício orçamentário financeiro responsável emylson farias silva relator conselheiro ronald polanco ribeiro plenário ementa prestação contas secretaria estado segurança pública sesp exercício regular com ressalva arquivamento dos autos vistos relatados discutidos autos acima identificado acordam membros à unanimidade nos termos voto conselheiro relator com fundamento artigo inciso da lei complementar estadual considerar regula com ressalva prestação contas secretaria estado segurança pública sesp referente exercício orçamentário financeiro responsabilidade sr emylson farias silva secretário segurança pública valendo como ressalvas concessões suprimentos fundos responsável por dois suprimentos concessão diárias posteriores aos deslocamentos